<a href="https://colab.research.google.com/github/ClauCll/Curso_ML_A_Z/blob/master/Regresi%C3%B3n/Regresion_Bajas_Antiguedad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Cuantas serán las bajas voluntarias de los clientes fijos para el periodo 202009. 
 

In [4]:
import pandas as pd
import numpy as np
from mpl_toolkits.mplot3d import axes3d
import matplotlib.pyplot as plt
import gspread
from oauth2client.client import GoogleCredentials 
from google.colab import auth
# Autoriza Claudia
auth.authenticate_user()
gc = gspread.authorize (GoogleCredentials.get_application_default())

### leemos datos

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
data = pd.read_csv('/content/drive/My Drive/Dataset/DBajas_ANTIGUEDAD.txt',delimiter='~')
X  = data.iloc[:,:-1].values
Y  = data.iloc[:,2].values
Y = Y[:,np.newaxis]

In [ ]:
i_año = [] # identificador de año
x_año = [] # año
x_mes = [] # mes

for _ in X[:,0]:
    if str(_)[0:4] == "2019":
        x_año.append(2019)
        x_mes.append(int(str(_)[4:7]))
        i_año.append(0) 
    if str(_)[0:4] == "2020":
        x_año.append(2020)
        x_mes.append(int(str(_)[4:7]))
        i_año.append(1)
i_año = np.array(i_año)[:,np.newaxis]
x_año = np.array(x_año)[:,np.newaxis]
x_mes = np.array(x_mes)[:,np.newaxis]


# Periodo - Años de antiguedad - Bajas
# 0 -> Negro
# 1 -> Rojo
# 2 -> Naranja
# 3 -> Amarillo 
# 4 -> Azul
# 5 -> Violeta 
# 6 -> Verde

i_año_ant = []
for _ in X[:,1]:
    if int(_) == 0:
        i_año_ant.append(0)
    if int(_) == 1:
        i_año_ant.append(1)
    if int(_) == 2:
        i_año_ant.append(2)
    if int(_) == 3:
        i_año_ant.append(3)
    if int(_) == 4:
        i_año_ant.append(4)
    if int(_) == 5:
        i_año_ant.append(5)
    if int(_) >  5:
        i_año_ant.append(6) 
i_año_ant = np.array(i_año_ant)[:,np.newaxis]

c0,c1,c2,c3 ,c4 ,c5 ,c6  = [],[],[],[],[],[],[]
c7,c8,c9,c10,c11,c12,c13 = [],[],[],[],[],[],[]
for _ in list(range(len(X[:,1]))):
    # 2019 ------------------------------------------------
    if i_año_ant[_,0] == 0 and i_año[_,0] == 0 :
        c0.append(1)
    else:
        c0.append(0)
    if i_año_ant[_,0] == 1 and i_año[_,0] == 0 : # 2019 - 1 año de antiguedad
        c1.append(1)
    else:
        c1.append(0)
    if i_año_ant[_,0] == 2 and i_año[_,0] == 0 :
        c2.append(1)
    else:
        c2.append(0)
    if i_año_ant[_,0] == 3 and i_año[_,0] == 0 :
        c3.append(1)
    else:
        c3.append(0)
    if i_año_ant[_,0] == 4 and i_año[_,0] == 0 :
        c4.append(1)
    else:
        c4.append(0)
    if i_año_ant[_,0] == 5 and i_año[_,0] == 0 :
        c5.append(1)
    else:
        c5.append(0)
    if i_año_ant[_,0] >= 6 and i_año[_,0] == 0 :
        c6.append(1)
    else:
        c6.append(0)
    # 2020 ------------------------------------------------
    if i_año_ant[_,0] == 0 and i_año[_,0] == 1 :
        c7.append(1)
    else:
        c7.append(0)
    if i_año_ant[_,0] == 1 and i_año[_,0] == 1 :
        c8.append(1)
    else:
        c8.append(0)
    if i_año_ant[_,0] == 2 and i_año[_,0] == 1 :
        c9.append(1)
    else:
        c9.append(0)
    if i_año_ant[_,0] == 3 and i_año[_,0] == 1 :
        c10.append(1)
    else:
        c10.append(0)
    if i_año_ant[_,0] == 4 and i_año[_,0] == 1 :
        c11.append(1)
    else:
        c11.append(0)
    if i_año_ant[_,0] == 5 and i_año[_,0] == 1 :
        c12.append(1)
    else:
        c12.append(0)
    if i_año_ant[_,0] >= 6 and i_año[_,0] == 1 :
        c13.append(1)
    else:
        c13.append(0)
        
c0  = np.array(c0 )[:,np.newaxis]
c1  = np.array(c1 )[:,np.newaxis]
c2  = np.array(c2 )[:,np.newaxis]
c3  = np.array(c3 )[:,np.newaxis]
c4  = np.array(c4 )[:,np.newaxis]
c5  = np.array(c5 )[:,np.newaxis]
c6  = np.array(c6 )[:,np.newaxis]
c7  = np.array(c7 )[:,np.newaxis]
c8  = np.array(c8 )[:,np.newaxis]
c9  = np.array(c9 )[:,np.newaxis]
c10 = np.array(c10)[:,np.newaxis]
c11 = np.array(c11)[:,np.newaxis]
c12 = np.array(c12)[:,np.newaxis]
c13 = np.array(c13)[:,np.newaxis]

plt.scatter(x_mes[ c0[:,0]==1 , 0 ], Y[ c0[:,0]==1 , 0 ], 
            c="black", marker='^', alpha=0.5) # 2019 -> i_año(0)
plt.scatter(x_mes[ c1[:,0]==1 , 0 ], Y[ c1[:,0]==1 , 0 ], 
            c="red", marker='^', alpha=0.5) # 2019 -> i_año(0)
plt.scatter(x_mes[ c2[:,0]==1 , 0 ], Y[ c2[:,0]==1 , 0 ], 
            c="orange", marker='^', alpha=0.5) # 2019 -> i_año(0)
plt.scatter(x_mes[ c3[:,0]==1 , 0 ], Y[ c3[:,0]==1 , 0 ], 
            c="salmon", marker='^', alpha=0.5) # 2019 -> i_año(0)
plt.scatter(x_mes[ c4[:,0]==1 , 0 ], Y[ c4[:,0]==1 , 0 ], 
            c="blue", marker='^', alpha=0.5) # 2019 -> i_año(0)
plt.scatter(x_mes[ c5[:,0]==1 , 0 ], Y[ c5[:,0]==1 , 0 ], 
            c="violet", marker='^', alpha=0.5) # 2019 -> i_año(0)
plt.scatter(x_mes[ c6[:,0]==1 , 0 ], Y[ c6[:,0]==1 , 0 ], 
            c="green", marker='^', alpha=0.5) # 2019 -> i_año(0)

plt.scatter(x_mes[ c7[:,0]==1 , 0 ], Y[ c7[:,0]==1 , 0 ], 
            c="black", marker='o', alpha=0.5) # 2020 -> i_año(1)
plt.scatter(x_mes[ c8[:,0]==1 , 0 ], Y[ c8[:,0]==1 , 0 ], 
            c="red", marker='o', alpha=0.5) # 2020 -> i_año(1)
plt.scatter(x_mes[ c9[:,0]==1 , 0 ], Y[ c9[:,0]==1 , 0 ], 
            c="orange", marker='o', alpha=0.5) # 2020 -> i_año(1)
plt.scatter(x_mes[ c10[:,0]==1 , 0 ], Y[ c10[:,0]==1 , 0 ], 
            c="salmon", marker='o', alpha=0.5) # 2020 -> i_año(1)
plt.scatter(x_mes[ c11[:,0]==1 , 0 ], Y[ c11[:,0]==1 , 0 ], 
            c="blue", marker='o', alpha=0.5) # 2020 -> i_año(1)
plt.scatter(x_mes[ c12[:,0]==1 , 0 ], Y[ c12[:,0]==1 , 0 ], 
            c="violet", marker='o', alpha=0.5) # 2020 -> i_año(1)
plt.scatter(x_mes[ c13[:,0]==1 , 0 ], Y[ c13[:,0]==1 , 0 ], 
            c="green", marker='o', alpha=0.5) # 2020 -> i_año(1)

# 0 -> Negro
# 1 -> Rojo
# 2 -> Naranja
# 3 -> Amarillo 
# 4 -> Azul
# 5 -> Violeta 
# 6 -> Verde

plt.xlabel('Mes')
plt.ylabel('Cantidad')
plt.title('Bajas por antiguedad')
plt.show()


plt.scatter(x_mes[ c0[:,0]==1 , 0 ], Y[ c0[:,0]==1 , 0 ], 
            c="black", marker='^', alpha=0.5) # 2019 -> i_año(0)

plt.scatter(x_mes[ c7[:,0]==1 , 0 ], Y[ c7[:,0]==1 , 0 ], 
            c="black", marker='o', alpha=0.5) # 2020 -> i_año(1)

plt.xlabel('Mes')
plt.ylabel('Cantidad')
plt.title('Bajas por antiguedad -0 años-')
plt.show()



plt.scatter(x_mes[ c1[:,0]==1 , 0 ], Y[ c1[:,0]==1 , 0 ], 
            c="red", marker='^', alpha=0.5) # 2019 -> i_año(0)

plt.scatter(x_mes[ c8[:,0]==1 , 0 ], Y[ c8[:,0]==1 , 0 ], 
            c="red", marker='o', alpha=0.5) # 2020 -> i_año(1)

plt.xlabel('Mes')
plt.ylabel('Cantidad')
plt.title('Bajas por antiguedad -1 años-')
plt.show()



plt.scatter(x_mes[ c2[:,0]==1 , 0 ], Y[ c2[:,0]==1 , 0 ], 
            c="orange", marker='^', alpha=0.5) # 2019 -> i_año(0)

plt.scatter(x_mes[ c9[:,0]==1 , 0 ], Y[ c9[:,0]==1 , 0 ], 
            c="orange", marker='o', alpha=0.5) # 2020 -> i_año(1)

plt.xlabel('Mes')
plt.ylabel('Cantidad')
plt.title('Bajas por antiguedad -2 años-')
plt.show()



plt.scatter(x_mes[ c3[:,0]==1 , 0 ], Y[ c3[:,0]==1 , 0 ], 
            c="salmon", marker='^', alpha=0.5) # 2019 -> i_año(0)

plt.scatter(x_mes[ c10[:,0]==1 , 0 ], Y[ c10[:,0]==1 , 0 ], 
            c="salmon", marker='o', alpha=0.5) # 2020 -> i_año(1)

plt.xlabel('Mes')
plt.ylabel('Cantidad')
plt.title('Bajas por antiguedad -3 años-')
plt.show()



plt.scatter(x_mes[ c4[:,0]==1 , 0 ], Y[ c4[:,0]==1 , 0 ], 
            c="blue", marker='^', alpha=0.5) # 2019 -> i_año(0)

plt.scatter(x_mes[ c11[:,0]==1 , 0 ], Y[ c11[:,0]==1 , 0 ], 
            c="blue", marker='o', alpha=0.5) # 2020 -> i_año(1)

plt.xlabel('Mes')
plt.ylabel('Cantidad')
plt.title('Bajas por antiguedad -4 años-')
plt.show()



plt.scatter(x_mes[ c5[:,0]==1 , 0 ], Y[ c5[:,0]==1 , 0 ], 
            c="violet", marker='^', alpha=0.5) # 2019 -> i_año(0)

plt.scatter(x_mes[ c12[:,0]==1 , 0 ], Y[ c12[:,0]==1 , 0 ], 
            c="violet", marker='o', alpha=0.5) # 2020 -> i_año(1)

plt.xlabel('Mes')
plt.ylabel('Cantidad')
plt.title('Bajas por antiguedad -5 años-')
plt.show()



plt.scatter(x_mes[ c6[:,0]==1 , 0 ], Y[ c6[:,0]==1 , 0 ], 
            c="green", marker='^', alpha=0.5) # 2019 -> i_año(0)

plt.scatter(x_mes[ c13[:,0]==1 , 0 ], Y[ c13[:,0]==1 , 0 ], 
            c="green", marker='o', alpha=0.5) # 2020 -> i_año(1)

plt.xlabel('Mes')
plt.ylabel('Cantidad')
plt.title('Bajas por antiguedad -mas de 5 años-')
plt.show()

## Modelo Polinómico

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures 
from sklearn.metrics import accuracy_score

In [ ]:
# 2019 - 1 año de antiguedad
x  = x_mes[ c1[:,0]==1 , 0 ]
y  = Y    [ c1[:,0]==1 , 0 ]

x  = x[:,np.newaxis]
y  = y[:,np.newaxis]
x_train, x_test, y_train, y_test = train_test_split(x, y, 
                                                    test_size = 0.3, 
                                                    random_state = 0)

gra_poly = 3 # grado del polinomio 

pf      = PolynomialFeatures(degree = gra_poly)    
X_poly  = pf.fit_transform(x_train.reshape(-1,1))  
regresion_lineal = LinearRegression() 
  
modelo1 = regresion_lineal.fit(X_poly, y_train) 

print("\n" + 'w = ' + str(modelo1.coef_) + ', b = ' + str(modelo1.intercept_))

# Train - Exactitud
x_ = x_train
y_ = y_train
x_ = pf.fit_transform(x_.reshape(-1,1))
y_predict_train = modelo1.predict(x_)
print("\n"+"2019 - score_train = "+str(modelo1.score(x_,y_))+"\n")

# Test - Exactitud
x__            = x_test
y__            = y_test
x__            = pf.fit_transform(x__.reshape(-1,1))
y_predict_test = modelo1.predict(x__)
print("2019 - score_test  = "+str(modelo1.score(x__,y__))+"\n")

plt.scatter(x_train, y_train, c='red', marker='^', alpha=0.5)
plt.scatter(x_test , y_test , c='red', marker='*', alpha=0.5)
plt.scatter(x_train, y_predict_train, c='green', marker='o', alpha=0.3)
plt.scatter(x_test , y_predict_test , c='green', marker='o', alpha=0.3)
plt.xlabel('Mes')
plt.ylabel('Cantidad')
plt.title('Bajas por antiguedad -2019 - 1 año de antiguedad-')
plt.show()

# 2020 - 1 año de antiguedad
x  = x_mes[ c8[:,0]==1 , 0 ]
y  = Y    [ c8[:,0]==1 , 0 ]

x  = x[:,np.newaxis]
y  = y[:,np.newaxis]
x_train, x_test, y_train, y_test = train_test_split(x, y, 
                                                    test_size = 0.3, 
                                                    random_state = 0)

gra_poly = 2 # grado del polinomio 

pf      = PolynomialFeatures(degree = gra_poly)    
X_poly  = pf.fit_transform(x_train.reshape(-1,1))  
regresion_lineal = LinearRegression() 
  
modelo2 = regresion_lineal.fit(X_poly, y_train) 

print("\n" + 'w = ' + str(modelo2.coef_) + ', b = ' + str(modelo2.intercept_))

# Train - Exactitud
x_ = x_train
y_ = y_train
x_ = pf.fit_transform(x_.reshape(-1,1))
y_predict_train = modelo2.predict(x_)
print("\n"+"2020 - score_train = "+str(modelo2.score(x_,y_))+"\n")

# Test - Exactitud
x__            = x_test
y__            = y_test
x__            = pf.fit_transform(x__.reshape(-1,1))
y_predict_test = modelo2.predict(x__)
print("2020 - score_test  = "+str(modelo2.score(x__,y__))+"\n")

plt.scatter(x_train, y_train, c='blue', marker='^', alpha=0.5)
plt.scatter(x_test , y_test , c='blue', marker='*', alpha=0.5)
plt.scatter(x_train, y_predict_train, c='black', marker='o', alpha=0.3)
plt.scatter(x_test , y_predict_test , c='black', marker='o', alpha=0.3)
plt.xlabel('Mes')
plt.ylabel('Cantidad')
plt.title('Bajas por antiguedad -2020 - 1 año de antiguedad-')


#% mes < agosto >
v_pr = [8] # valor a predecir

v_pr = np.array(v_pr)
v_pr = v_pr[:,np.newaxis]
x_pr = pf.fit_transform(v_pr.reshape(-1,1))

# y_pr1= modelo1.predict(x_pr)
y_pr2= modelo2.predict(x_pr)
print("\n"+"La cantidad de bajas para el mes "+str(v_pr[0][0])+
      " de 2020 es -> "+str(y_pr2[0][0]))

plt.scatter(v_pr, y_pr2, c='violet', marker='o', alpha=1)
plt.show()

# 2019 - 1 año de antiguedad
x  = x_mes[ c1[:,0]==1 , 0 ]
y  = Y    [ c1[:,0]==1 , 0 ]

x  = x[:,np.newaxis]
y  = y[:,np.newaxis]
x_train, x_test, y_train, y_test = train_test_split(x, y, 
                                                    test_size = 0.3, 
                                                    random_state = 0)

gra_poly = 3 # grado del polinomio 

pf      = PolynomialFeatures(degree = gra_poly)    
X_poly  = pf.fit_transform(x_train.reshape(-1,1))  
regresion_lineal = LinearRegression() 
  
modelo1 = regresion_lineal.fit(X_poly, y_train) 

print("\n" + 'w = ' + str(modelo1.coef_) + ', b = ' + str(modelo1.intercept_))

# Train - Exactitud
x_ = x_train
y_ = y_train
x_ = pf.fit_transform(x_.reshape(-1,1))
y_predict_train = modelo1.predict(x_)
print("\n"+"2019 - score_train = "+str(modelo1.score(x_,y_))+"\n")

# Test - Exactitud
x__            = x_test
y__            = y_test
x__            = pf.fit_transform(x__.reshape(-1,1))
y_predict_test = modelo1.predict(x__)
print("2019 - score_test  = "+str(modelo1.score(x__,y__))+"\n")

plt.scatter(x_train, y_train, c='red', marker='^', alpha=0.5)
plt.scatter(x_test , y_test , c='red', marker='*', alpha=0.5)
plt.scatter(x_train, y_predict_train, c='green', marker='o', alpha=0.3)
plt.scatter(x_test , y_predict_test , c='green', marker='o', alpha=0.3)


# 2020 - 1 año de antiguedad
x  = x_mes[ c8[:,0]==1 , 0 ]
y  = Y    [ c8[:,0]==1 , 0 ]

x  = x[:,np.newaxis]
y  = y[:,np.newaxis]
x_train, x_test, y_train, y_test = train_test_split(x, y, 
                                                    test_size = 0.3, 
                                                    random_state = 0)

gra_poly = 2 # grado del polinomio 

pf      = PolynomialFeatures(degree = gra_poly)    
X_poly  = pf.fit_transform(x_train.reshape(-1,1))  
regresion_lineal = LinearRegression() 
  
modelo2 = regresion_lineal.fit(X_poly, y_train) 

print("\n" + 'w = ' + str(modelo2.coef_) + ', b = ' + str(modelo2.intercept_))

# Train - Exactitud
x_ = x_train
y_ = y_train
x_ = pf.fit_transform(x_.reshape(-1,1))
y_predict_train = modelo2.predict(x_)
print("\n"+"2020 - score_train = "+str(modelo2.score(x_,y_))+"\n")

# Test - Exactitud
x__            = x_test
y__            = y_test
x__            = pf.fit_transform(x__.reshape(-1,1))
y_predict_test = modelo2.predict(x__)
print("2020 - score_test  = "+str(modelo2.score(x__,y__))+"\n")

plt.scatter(x_train, y_train, c='blue', marker='^', alpha=0.5)
plt.scatter(x_test , y_test , c='blue', marker='*', alpha=0.5)
plt.scatter(x_train, y_predict_train, c='black', marker='o', alpha=0.3)
plt.scatter(x_test , y_predict_test , c='black', marker='o', alpha=0.3)
plt.xlabel('Mes')
plt.ylabel('Cantidad')
plt.title('Bajas por antiguedad -2019,2020 - 1 año de antiguedad-')


#% mes < agosto >
v_pr = [8] # valor a predecir

v_pr = np.array(v_pr)
v_pr = v_pr[:,np.newaxis]
x_pr = pf.fit_transform(v_pr.reshape(-1,1))

# y_pr1= modelo1.predict(x_pr)
y_pr2= modelo2.predict(x_pr)
print("\n"+"La cantidad de bajas para el mes "+str(v_pr[0][0])+
      " de 2020 es -> "+str(y_pr2[0][0]))

plt.scatter(v_pr, y_pr2, c='violet', marker='o', alpha=1)

#% mes < septiembre >
v_pr = [9] # valor a predecir

v_pr = np.array(v_pr)
v_pr = v_pr[:,np.newaxis]
x_pr = pf.fit_transform(v_pr.reshape(-1,1))

# y_pr1= modelo1.predict(x_pr)
y_pr2= modelo2.predict(x_pr)
print("\n"+"La cantidad de bajas para el mes "+str(v_pr[0][0])+
      " de 2020 es -> "+str(y_pr2[0][0]))

plt.scatter(v_pr, y_pr2, c='violet', marker='o', alpha=1)

#% mes < octubre >
v_pr = [10] # valor a predecir

v_pr = np.array(v_pr)
v_pr = v_pr[:,np.newaxis]
x_pr = pf.fit_transform(v_pr.reshape(-1,1))

# y_pr1= modelo1.predict(x_pr)
y_pr2= modelo2.predict(x_pr)
print("\n"+"La cantidad de bajas para el mes "+str(v_pr[0][0])+
      " de 2020 es -> "+str(y_pr2[0][0]))

plt.scatter(v_pr, y_pr2, c='violet', marker='o', alpha=1)

#% mes < noviembre >
v_pr = [11] # valor a predecir

v_pr = np.array(v_pr)
v_pr = v_pr[:,np.newaxis]
x_pr = pf.fit_transform(v_pr.reshape(-1,1))

# y_pr1= modelo1.predict(x_pr)
y_pr2= modelo2.predict(x_pr)
print("\n"+"La cantidad de bajas para el mes "+str(v_pr[0][0])+
      " de 2020 es -> "+str(y_pr2[0][0]))

plt.scatter(v_pr, y_pr2, c='violet', marker='o', alpha=1)

#% mes < diciembre >
v_pr = [12] # valor a predecir

v_pr = np.array(v_pr)
v_pr = v_pr[:,np.newaxis]
x_pr = pf.fit_transform(v_pr.reshape(-1,1))

# y_pr1= modelo1.predict(x_pr)
y_pr2= modelo2.predict(x_pr)
print("\n"+"La cantidad de bajas para el mes "+str(v_pr[0][0])+
      " de 2020 es -> "+str(y_pr2[0][0]))

plt.scatter(v_pr, y_pr2, c='violet', marker='o', alpha=1)
plt.show()
